<a href="https://colab.research.google.com/github/Esther-Opeyemi/Activity1/blob/master/StageC_CL_Stability_of_the_Grid_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import important libraries
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.utils
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from xgboost import XGBClassifier

In [8]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [10]:
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [11]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [12]:
# shuffle and assign inputs and outputs for the model
df1 = shuffle(df)
df1 = df1.reset_index(drop=True)

inputs = df1.drop(columns=['stabf', 'stab'])
outputs = df1['stabf']

In [16]:
# split data
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=1)

In [17]:
x_test.reset_index(drop=True)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,6.987524,8.374817,9.098630,0.681482,2.531060,-0.570890,-1.239336,-0.720833,0.682980,0.506989,0.299548,0.258607
1,0.732237,6.457205,2.519639,7.467276,4.089326,-1.621364,-0.552015,-1.915947,0.411778,0.936109,0.420461,0.363010
2,2.211061,9.851117,6.775547,4.306293,4.027788,-1.677891,-0.513667,-1.836230,0.801062,0.346599,0.174836,0.973675
3,9.691965,4.898546,0.802097,8.451251,3.834120,-1.396450,-0.725575,-1.712095,0.577103,0.156035,0.777497,0.296236
4,8.641240,0.724772,9.419006,7.697809,4.589777,-1.626768,-0.980922,-1.982088,0.698569,0.671294,0.446843,0.670736
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.841976,6.475664,8.133568,0.875038,4.537393,-1.422611,-1.957640,-1.157141,0.360198,0.543758,0.723875,0.232706
1996,6.349374,2.394443,3.645365,3.373972,5.004762,-1.735476,-1.317382,-1.951904,0.818224,0.361062,0.772124,0.462628
1997,5.431413,7.132318,3.244979,9.197791,3.535651,-1.141194,-0.576927,-1.817529,0.382137,0.373259,0.646594,0.491825
1998,3.694493,9.271351,5.917965,9.108992,3.733560,-0.611769,-1.286072,-1.835719,0.781086,0.098976,0.620089,0.981051


In [18]:
# scale dataset
scaler = StandardScaler()

X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [20]:
# train model with random forest
rf_class = RandomForestClassifier(random_state=1)
rf_class.fit(X_train, y_train)

rf_class_p = rf_class.predict(X_test)

In [21]:
# confusion_matrix
conf_matrix = confusion_matrix(y_true=y_test, y_pred=rf_class_p, labels=['stable', 'unstable'])
conf_matrix

array([[ 621,  115],
       [  61, 1203]])

In [22]:
accuracy = accuracy_score(y_true=y_test, y_pred=rf_class_p)
print(f'accuracy: {accuracy*100}')

accuracy: 91.2


In [23]:
et_class = ExtraTreesClassifier(random_state=1)
et_class.fit(X_train, y_train)

et_class_p = et_class.predict(X_test)

In [24]:
accuracy = accuracy_score(y_true=y_test, y_pred=et_class_p)
print(f'accuracy: {accuracy*100}')

accuracy: 92.0


In [25]:
et_class1 = ExtraTreesClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=8, max_features=None, random_state=1)
et_class1.fit(X_train, y_train)

et_class1 = et_class1.predict(X_test)

In [26]:
accuracy1 = accuracy_score(y_true=y_test, y_pred=et_class1)
print(f'accuracy: {accuracy*100}')

accuracy: 92.0
